In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Reply.csv to Reply.csv


In [ ]:
import io
import pandas as pd
df2 = pd.read_csv(io.BytesIO(uploaded['Reply.csv']))
# Dataset is now stored in a Pandas Dataframe

In [ ]:
df2.head()

,Reply mail
0,traveling to have a business meeting takes the...
1,greg how about either next tuesday or thursday...
2,any morning between and
3,paula million is fine phillip
4,i think fletch has a good cpa i am still doin...


In [ ]:
df2.shape

(9399, 1)

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import string

In [ ]:
df2['Reply mail'][1]

'greg how about either next tuesday or thursday  phillip'

In [ ]:
len(df2['Reply mail'][1])

55

In [ ]:
need_list=[]
rested_list=[]

for i in df2['Reply mail']:
  j=i.split()
  if len(j)>20:
    needed=j[:20]
    rest=j[20:]
    need_list.append(' '.join(needed))
    rested_list.append(' '.join(rest))




In [ ]:
len(need_list)

7851

In [ ]:
len(rested_list)

7851

In [ ]:
need_list[0]

'traveling to have a business meeting takes the fun out of the trip especially if you have to prepare a'

In [ ]:
rested_list[0]

'presentation i would suggest holding the business plan meetings here then take a trip without any formal business meetings i would even try and get some honest opinions on whether a trip is even desired or necessary as far as the business meetings i think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not too often the presenter speaks and the others are quiet just waiting for their turn the meetings might be better if held in a round table discussion format my suggestion for where to go is austin play golf and rent a ski boat and jet ski s flying somewhere takes too much time'

In [ ]:
X_train2 = need_list[:6000]

In [ ]:
X_test2 = need_list[6000:]

In [ ]:
len(X_train2)

6000

In [ ]:
X_train2[0]

'traveling to have a business meeting takes the fun out of the trip especially if you have to prepare a'

In [ ]:
len(X_test2)

1851

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
tokenizer3 = Tokenizer()
tokenizer3.fit_on_texts(X_train2)
sequences3 = tokenizer3.texts_to_sequences(X_train2)

In [ ]:
len(sequences3)

6000

In [ ]:
sequences3 = np.array(sequences3)


In [ ]:
X1,y1 = sequences3[:,:-1], sequences3[:,-1]

In [ ]:
vocab_size3 = len(tokenizer3.word_index)+1

In [ ]:
X1[0]

array([1920,    1,   22,    9,  337,   86, 1506,    2,  318,   69,   14,
          2,  456, 1058,   46,    4,   22,    1, 1685])

In [ ]:
y1 = to_categorical(y1,num_classes=vocab_size3)

In [ ]:
seq_length = X1.shape[1]

In [ ]:
model2 = Sequential()
model2.add(Embedding(vocab_size3,1,input_length=seq_length))
model2.add(LSTM(100,return_sequences=True))
model2.add(LSTM(100))
model2.add(Dense(100,activation='relu'))
model2.add(Dense(vocab_size3,activation='softmax'))


In [ ]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 19, 1)             7284      
_________________________________________________________________
lstm (LSTM)                  (None, 19, 100)           40800     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 7284)              735684    
Total params: 874,268
Trainable params: 874,268
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model2.fit(X1,y1,batch_size=256,epochs=200)

Epoch 1/200
24/24 [==============================] - 9s 19ms/step - loss: 8.7982 - accuracy: 0.0232
Epoch 2/200
24/24 [==============================] - 0s 15ms/step - loss: 6.4226 - accuracy: 0.0317
Epoch 3/200
24/24 [==============================] - 0s 14ms/step - loss: 6.1419 - accuracy: 0.0328
Epoch 4/200
24/24 [==============================] - 0s 14ms/step - loss: 6.0463 - accuracy: 0.0342
Epoch 5/200
24/24 [==============================] - 0s 13ms/step - loss: 6.0229 - accuracy: 0.0311
Epoch 6/200
24/24 [==============================] - 0s 13ms/step - loss: 6.0210 - accuracy: 0.0349
Epoch 7/200
24/24 [==============================] - 0s 13ms/step - loss: 6.0024 - accuracy: 0.0370
Epoch 8/200
24/24 [==============================] - 0s 13ms/step - loss: 6.0114 - accuracy: 0.0389
Epoch 9/200
24/24 [==============================] - 0s 13ms/step - loss: 6.0245 - accuracy: 0.0342
Epoch 10/200
24/24 [==============================] - 0s 13ms/step - loss: 6.0332 - accuracy: 0.0339

In [ ]:
def generate_text_seq(model,tokenizer,text_seq_length, seed_text,n_words):
  text=[]

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded],maxlen=text_seq_length,truncating='pre')

    y_predict = model.predict_classes(encoded)
    predicted_word =''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)  


In [ ]:
len(y_test)


1851

In [ ]:
y_test=rested_list[6000:]

In [ ]:
y_test[123]

'bond credits sdge recently refunded these credits to our customers by check we are currently in the process of collecting these credits from the commercial accounts but do we recover these credits amounting to from the residential customers below is a description of the reason for this credit this credit has been made possible due to a unique set of circumstances involving electric industry restructuring which has proceeded in accordance with california state law ab ab required sdg e to lower electric rates by for residential and small commercial customers then froze rate levels for up to years ab also allowed sdg e to issue rate reduction bonds to finance the rate reduction in sdg e issued rate reduction bonds in an amount adequate to provide a rate reduction for years however sdg e was able to recover many costs related to the transition to a competitive market years early by last july through the sale of its two san diego power plants at higher than expected prices and through nume

In [ ]:
X_test2[123]

'all this is currently an issue not only with the annual merger savings credit but also the sdge rate reduction'

In [ ]:
seed_text=X_test2[123]

In [ ]:
generate_text_seq(model2,tokenizer3,seq_length, seed_text, 5 )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'monday gas over cwhite wood'

In [ ]:
X_test2[213]

'steve peace is watching his political career vanish before his very eyes couldn t happen to a nicer guy rightly'

In [ ]:
y_test[213]

'or wrongly he s determined that this tack is his last and best option to salvage something whether anyone joins his parade is a whole other question nevertheless we should expect him to continue susan j mara am to james d steffes na enron enron paul kaufman pdx ect ect richar d shapiro na enron enron jeff dasovich na enron enron mona l petrochko na enron enron mary hain hou ect ect joe hartsoe corp enron enron sarah novosel corp enron enron sandra mccubbin na enron enron rcarroll bracepatt com cc subject peace press conference notes this is pretty much an all out attack against the positions that we and the generators have taken forwarded by susan j mara na enron on am julee malinowski ball jmball ns net pm please respond to julee malinowski ball to william hall wfhall duke energy com trond aschehoug taschehoug thermoecotek com tony wetzel twetzel thermoecotek com tandy mcmannes mcmannes aol com sue mara smara enron com steven kelley steven iepa com steve ponder steve ponder fpl com r

In [ ]:
seed_text2=X_test2[213]

In [ ]:
generate_text_seq(model2,tokenizer3,seq_length, seed_text2, 5 )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'statement either months hour impacted'

In [ ]:
X_test2[321]

'greetings since i get everything that you forward you may remove me from you distribution list hope all is well'

In [ ]:
y_test[321]

'best jeff'

In [ ]:
seed_text3=X_test2[321]

In [ ]:
generate_text_seq(model2,tokenizer3,seq_length, seed_text3, 5 )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'we a fried of will'